# Energy: Delta Liquid Clustering Demo


## Overview


This notebook demonstrates the power of **Delta Liquid Clustering** in Oracle AI Data Platform (AIDP) Workbench using an energy and utilities analytics use case. Liquid clustering automatically optimizes data layout for query performance without requiring manual partitioning or Z-Ordering.

### What is Liquid Clustering?

Liquid clustering automatically identifies and groups similar data together based on clustering columns you define. This optimization happens automatically during data ingestion and maintenance operations, providing:

- **Automatic optimization**: No manual tuning required
- **Improved query performance**: Faster queries on clustered columns
- **Reduced maintenance**: No need for manual repartitioning
- **Adaptive clustering**: Adjusts as data patterns change

### Use Case: Smart Grid Monitoring and Energy Consumption Analytics

We'll analyze energy consumption and smart grid performance data. Our clustering strategy will optimize for:

- **Meter-specific queries**: Fast lookups by meter ID
- **Time-based analysis**: Efficient filtering by reading date and time
- **Consumption patterns**: Quick aggregation by location and energy type

### AIDP Environment Setup

This notebook leverages the existing Spark session in your AIDP environment.

In [ ]:
# Create energy catalog and analytics schema

# In AIDP, catalogs provide data isolation and governance

spark.sql("CREATE CATALOG IF NOT EXISTS energy")

spark.sql("CREATE SCHEMA IF NOT EXISTS energy.analytics")

print("Energy catalog and analytics schema created successfully!")

Energy catalog and analytics schema created successfully!


## Step 2: Create Delta Table with Liquid Clustering

### Table Design

Our `energy_readings` table will store:

- **meter_id**: Unique smart meter identifier
- **reading_date**: Date and time of meter reading
- **energy_type**: Type (Electricity, Gas, Water, Solar)
- **consumption**: Energy consumed (kWh, therms, gallons)
- **location**: Geographic location/region
- **peak_demand**: Peak usage during interval
- **efficiency_rating**: System efficiency (0-100)

### Clustering Strategy

We'll cluster by `meter_id` and `reading_date` because:

- **meter_id**: Meters generate regular readings, grouping consumption history together
- **reading_date**: Time-based queries are critical for billing cycles, demand analysis, and seasonal patterns
- This combination optimizes for both meter monitoring and temporal energy consumption analysis

In [ ]:
# Create Delta table with liquid clustering

# CLUSTER BY defines the columns for automatic optimization

spark.sql("""

CREATE TABLE IF NOT EXISTS energy.analytics.energy_readings (

    meter_id STRING,

    reading_date TIMESTAMP,

    energy_type STRING,

    consumption DECIMAL(10,3),

    location STRING,

    peak_demand DECIMAL(8,2),

    efficiency_rating INT

)

USING DELTA

CLUSTER BY (meter_id, reading_date)

""")

print("Delta table with liquid clustering created successfully!")

print("Clustering will automatically optimize data layout for queries on meter_id and reading_date.")

Delta table with liquid clustering created successfully!
Clustering will automatically optimize data layout for queries on meter_id and reading_date.


## Step 3: Generate Energy Sample Data

### Data Generation Strategy

We'll create realistic energy consumption data including:

- **2,000 smart meters** with hourly readings over time
- **Energy types**: Electricity, Natural Gas, Water, Solar generation
- **Realistic consumption patterns**: Seasonal variations, peak usage times, efficiency differences
- **Geographic diversity**: Different locations with varying consumption profiles

### Why This Data Pattern?

This data simulates real energy scenarios where:

- Consumption varies by time of day and season
- Peak demand impacts grid stability
- Efficiency ratings affect sustainability goals
- Geographic patterns drive infrastructure planning
- Real-time monitoring enables demand response programs

In [ ]:
# Generate sample energy consumption data

# Using fully qualified imports to avoid conflicts

import random

from datetime import datetime, timedelta


# Define energy data constants

ENERGY_TYPES = ['Electricity', 'Natural Gas', 'Water', 'Solar']

LOCATIONS = ['Residential_NYC', 'Commercial_CHI', 'Industrial_HOU', 'Residential_LAX', 'Commercial_SFO']

# Base consumption parameters by energy type and location

CONSUMPTION_PARAMS = {

    'Electricity': {

        'Residential_NYC': {'base_consumption': 15, 'peak_factor': 2.5, 'efficiency': 85},

        'Commercial_CHI': {'base_consumption': 150, 'peak_factor': 3.0, 'efficiency': 78},

        'Industrial_HOU': {'base_consumption': 500, 'peak_factor': 2.2, 'efficiency': 92},

        'Residential_LAX': {'base_consumption': 12, 'peak_factor': 2.8, 'efficiency': 88},

        'Commercial_SFO': {'base_consumption': 180, 'peak_factor': 2.7, 'efficiency': 82}

    },

    'Natural Gas': {

        'Residential_NYC': {'base_consumption': 25, 'peak_factor': 1.8, 'efficiency': 90},

        'Commercial_CHI': {'base_consumption': 80, 'peak_factor': 2.1, 'efficiency': 85},

        'Industrial_HOU': {'base_consumption': 200, 'peak_factor': 1.9, 'efficiency': 95},

        'Residential_LAX': {'base_consumption': 20, 'peak_factor': 2.0, 'efficiency': 87},

        'Commercial_SFO': {'base_consumption': 95, 'peak_factor': 2.3, 'efficiency': 83}

    },

    'Water': {

        'Residential_NYC': {'base_consumption': 180, 'peak_factor': 1.5, 'efficiency': 88},

        'Commercial_CHI': {'base_consumption': 450, 'peak_factor': 1.7, 'efficiency': 82},

        'Industrial_HOU': {'base_consumption': 1200, 'peak_factor': 1.6, 'efficiency': 91},

        'Residential_LAX': {'base_consumption': 160, 'peak_factor': 1.8, 'efficiency': 85},

        'Commercial_SFO': {'base_consumption': 380, 'peak_factor': 1.9, 'efficiency': 79}

    },

    'Solar': {

        'Residential_NYC': {'base_consumption': -8, 'peak_factor': 3.5, 'efficiency': 78},

        'Commercial_CHI': {'base_consumption': -75, 'peak_factor': 4.0, 'efficiency': 85},

        'Industrial_HOU': {'base_consumption': -250, 'peak_factor': 3.8, 'efficiency': 88},

        'Residential_LAX': {'base_consumption': -12, 'peak_factor': 4.2, 'efficiency': 82},

        'Commercial_SFO': {'base_consumption': -95, 'peak_factor': 3.9, 'efficiency': 86}

    }

}


# Generate energy reading records

reading_data = []

base_date = datetime(2024, 1, 1)


# Create 2,000 meters with hourly readings for 3 months

for meter_num in range(1, 2001):

    meter_id = f"MTR{meter_num:06d}"
    
    # Each meter gets readings for 90 days (hourly)

    for day in range(90):

        for hour in range(24):

            reading_date = base_date + timedelta(days=day, hours=hour)
            
            # Select energy type and location for this meter

            energy_type = random.choice(ENERGY_TYPES)

            location = random.choice(LOCATIONS)
            
            params = CONSUMPTION_PARAMS[energy_type][location]
            
            # Calculate consumption with time-based variations

            # Seasonal variation (higher in winter for heating, summer for cooling)

            month = reading_date.month

            if energy_type in ['Electricity', 'Natural Gas']:

                if month in [12, 1, 2]:  # Winter

                    seasonal_factor = 1.4

                elif month in [6, 7, 8]:  # Summer

                    seasonal_factor = 1.3

                else:

                    seasonal_factor = 1.0

            else:

                seasonal_factor = 1.0
            
            # Time-of-day variation

            hour_factor = 1.0

            if hour in [6, 7, 8, 17, 18, 19]:  # Peak hours

                hour_factor = params['peak_factor']

            elif hour in [2, 3, 4, 5]:  # Off-peak

                hour_factor = 0.4

            
            # Calculate consumption

            consumption_variation = random.uniform(0.8, 1.2)

            consumption = round(params['base_consumption'] * seasonal_factor * hour_factor * consumption_variation, 3)
            
            # Peak demand (higher during peak hours)

            peak_demand = round(abs(consumption) * random.uniform(1.1, 1.5), 2)
            
            # Efficiency rating with some variation

            efficiency_variation = random.randint(-5, 3)

            efficiency_rating = max(0, min(100, params['efficiency'] + efficiency_variation))
            
            reading_data.append({

                "meter_id": meter_id,

                "reading_date": reading_date,

                "energy_type": energy_type,

                "consumption": consumption,

                "location": location,

                "peak_demand": peak_demand,

                "efficiency_rating": efficiency_rating

            })



print(f"Generated {len(reading_data)} energy reading records")

print("Sample record:", reading_data[0])

Generated 4320000 energy reading records
Sample record: {'meter_id': 'MTR000001', 'reading_date': datetime.datetime(2024, 1, 1, 0, 0), 'energy_type': 'Solar', 'consumption': -8.397, 'location': 'Residential_NYC', 'peak_demand': 11.81, 'efficiency_rating': 80}


## Step 4: Insert Data Using PySpark

### Data Insertion Strategy

We'll use PySpark to:

1. **Create DataFrame** from our generated data
2. **Insert into Delta table** with liquid clustering
3. **Verify the insertion** with a sample query

### Why PySpark for Insertion?

- **Distributed processing**: Handles large datasets efficiently
- **Type safety**: Ensures data integrity
- **Optimization**: Leverages Spark's query optimization
- **Liquid clustering**: Automatically applies clustering during insertion

In [ ]:
# Insert data using PySpark DataFrame operations

# Using fully qualified function references to avoid conflicts


# Create DataFrame from generated data

df_readings = spark.createDataFrame(reading_data)


# Display schema and sample data

print("DataFrame Schema:")

df_readings.printSchema()



print("\nSample Data:")

df_readings.show(5)


# Insert data into Delta table with liquid clustering

# The CLUSTER BY (meter_id, reading_date) will automatically optimize the data layout

df_readings.write.mode("overwrite").saveAsTable("energy.analytics.energy_readings")


print(f"\nSuccessfully inserted {df_readings.count()} records into energy.analytics.energy_readings")

print("Liquid clustering automatically optimized the data layout during insertion!")

DataFrame Schema:
root
 |-- consumption: double (nullable = true)
 |-- efficiency_rating: long (nullable = true)
 |-- energy_type: string (nullable = true)
 |-- location: string (nullable = true)
 |-- meter_id: string (nullable = true)
 |-- peak_demand: double (nullable = true)
 |-- reading_date: timestamp (nullable = true)


Sample Data:


+-----------+-----------------+-----------+---------------+---------+-----------+-------------------+
|consumption|efficiency_rating|energy_type|       location| meter_id|peak_demand|       reading_date|
+-----------+-----------------+-----------+---------------+---------+-----------+-------------------+
|     -8.397|               80|      Solar|Residential_NYC|MTR000001|      11.81|2024-01-01 00:00:00|
|    -13.923|               83|      Solar|Residential_LAX|MTR000001|      18.41|2024-01-01 01:00:00|
|    113.538|               83|Electricity| Commercial_SFO|MTR000001|     125.27|2024-01-01 02:00:00|
|    145.708|               78|      Water| Commercial_CHI|MTR000001|     196.87|2024-01-01 03:00:00|
|    489.841|               86|      Water| Industrial_HOU|MTR000001|     611.02|2024-01-01 04:00:00|
+-----------+-----------------+-----------+---------------+---------+-----------+-------------------+
only showing top 5 rows




Successfully inserted 4320000 records into energy.analytics.energy_readings
Liquid clustering automatically optimized the data layout during insertion!


## Step 5: Demonstrate Liquid Clustering Benefits

### Query Performance Analysis

Now let's see how liquid clustering improves query performance. We'll run queries that benefit from our clustering strategy:

1. **Meter reading history** (clustered by meter_id)
2. **Time-based consumption analysis** (clustered by reading_date)
3. **Combined meter + time queries** (optimal for our clustering)

### Expected Performance Benefits

With liquid clustering, these queries should be significantly faster because:

- **Data locality**: Related records are physically grouped together
- **Reduced I/O**: Less data needs to be read from disk
- **Automatic optimization**: No manual tuning required

In [ ]:
# Demonstrate liquid clustering benefits with optimized queries


# Query 1: Meter reading history - benefits from meter_id clustering

print("=== Query 1: Meter Reading History ===")

meter_history = spark.sql("""

SELECT meter_id, reading_date, energy_type, consumption, peak_demand, efficiency_rating

FROM energy.analytics.energy_readings

WHERE meter_id = 'MTR000001'

ORDER BY reading_date DESC

LIMIT 24

""")



meter_history.show()

print(f"Records found: {meter_history.count()}")



# Query 2: Time-based peak demand analysis - benefits from reading_date clustering

print("\n=== Query 2: Recent Peak Demand Issues ===")

peak_demand = spark.sql("""

SELECT reading_date, meter_id, location, peak_demand, energy_type

FROM energy.analytics.energy_readings

WHERE DATE(reading_date) = '2024-02-15' AND peak_demand > 200

ORDER BY peak_demand DESC

""")



peak_demand.show()

print(f"Peak demand issues found: {peak_demand.count()}")



# Query 3: Combined meter + time query - optimal for our clustering strategy

print("\n=== Query 3: Meter Consumption Trends ===")

consumption_trends = spark.sql("""

SELECT meter_id, reading_date, energy_type, consumption, efficiency_rating

FROM energy.analytics.energy_readings

WHERE meter_id LIKE 'MTR000%' AND reading_date >= '2024-02-01'

ORDER BY meter_id, reading_date

LIMIT 50

""")



consumption_trends.show()

print(f"Consumption trend records found: {consumption_trends.count()}")

=== Query 1: Meter Reading History ===


+---------+-------------------+-----------+-----------+-----------+-----------------+
| meter_id|       reading_date|energy_type|consumption|peak_demand|efficiency_rating|
+---------+-------------------+-----------+-----------+-----------+-----------------+
|MTR000001|2024-03-30 23:00:00|      Solar|    -76.497|     100.24|               83|
|MTR000001|2024-03-30 22:00:00|      Water|   1110.183|    1612.46|               89|
|MTR000001|2024-03-30 21:00:00|Natural Gas|     20.917|       24.2|               88|
|MTR000001|2024-03-30 20:00:00|      Water|   1129.645|    1513.78|               92|
|MTR000001|2024-03-30 19:00:00|      Solar|   -311.465|     355.24|               81|
|MTR000001|2024-03-30 18:00:00|Electricity|    1126.97|    1515.54|               92|
|MTR000001|2024-03-30 17:00:00|      Water|   2149.727|    2591.09|               88|
|MTR000001|2024-03-30 16:00:00|Electricity|    188.143|      262.5|               84|
|MTR000001|2024-03-30 15:00:00|Electricity|    579.727

Records found: 24

=== Query 2: Recent Peak Demand Issues ===


+-------------------+---------+--------------+-----------+-----------+
|       reading_date| meter_id|      location|peak_demand|energy_type|
+-------------------+---------+--------------+-----------+-----------+
|2024-02-15 19:00:00|MTR000069|Industrial_HOU|    3390.26|      Water|
|2024-02-15 18:00:00|MTR001732|Industrial_HOU|    3384.62|      Water|
|2024-02-15 17:00:00|MTR001502|Industrial_HOU|    3349.98|      Water|
|2024-02-15 17:00:00|MTR000428|Industrial_HOU|    3312.01|      Water|
|2024-02-15 19:00:00|MTR001003|Industrial_HOU|    3282.39|      Water|
|2024-02-15 17:00:00|MTR000272|Industrial_HOU|    3274.09|      Water|
|2024-02-15 06:00:00|MTR000513|Industrial_HOU|    3273.61|      Water|
|2024-02-15 06:00:00|MTR000856|Industrial_HOU|     3258.9|      Water|
|2024-02-15 19:00:00|MTR000552|Industrial_HOU|    3237.69|      Water|
|2024-02-15 19:00:00|MTR000486|Industrial_HOU|    3231.59|      Water|
|2024-02-15 07:00:00|MTR001437|Industrial_HOU|    3226.26|      Water|
|2024-

Peak demand issues found: 23244

=== Query 3: Meter Consumption Trends ===


+---------+-------------------+-----------+-----------+-----------------+
| meter_id|       reading_date|energy_type|consumption|efficiency_rating|
+---------+-------------------+-----------+-----------+-----------------+
|MTR000001|2024-02-01 00:00:00|Electricity|     19.291|               85|
|MTR000001|2024-02-01 01:00:00|Electricity|     18.316|               82|
|MTR000001|2024-02-01 02:00:00|Natural Gas|     45.141|               81|
|MTR000001|2024-02-01 03:00:00|Natural Gas|     12.816|               88|
|MTR000001|2024-02-01 04:00:00|      Water|    413.259|               88|
|MTR000001|2024-02-01 05:00:00|      Water|    124.545|               82|
|MTR000001|2024-02-01 06:00:00|Electricity|     59.509|               84|
|MTR000001|2024-02-01 07:00:00|Natural Gas|     267.85|               81|
|MTR000001|2024-02-01 08:00:00|Electricity|    597.628|               77|
|MTR000001|2024-02-01 09:00:00|Natural Gas|     32.049|               85|
|MTR000001|2024-02-01 10:00:00|      S

Consumption trend records found: 50


## Step 6: Analyze Clustering Effectiveness

### Understanding the Impact

Let's examine how liquid clustering has organized our data and analyze some aggregate statistics to demonstrate the energy insights possible with this optimized structure.

### Key Analytics

- **Meter performance** and consumption patterns
- **Location-based energy usage** and demand analysis
- **Energy type efficiency** and sustainability metrics
- **Peak demand patterns** and grid optimization

In [ ]:
# Analyze clustering effectiveness and energy insights


# Meter performance analysis

print("=== Meter Performance Analysis ===")

meter_performance = spark.sql("""

SELECT meter_id, COUNT(*) as total_readings,

       ROUND(AVG(consumption), 3) as avg_consumption,

       ROUND(MAX(peak_demand), 2) as max_peak_demand,

       ROUND(AVG(efficiency_rating), 2) as avg_efficiency,

       ROUND(SUM(ABS(consumption)), 3) as total_absolute_consumption

FROM energy.analytics.energy_readings

GROUP BY meter_id

ORDER BY total_absolute_consumption DESC

LIMIT 10

""")



meter_performance.show()


# Location-based consumption analysis

print("\n=== Location-Based Consumption Analysis ===")

location_analysis = spark.sql("""

SELECT location, COUNT(*) as total_readings,

       ROUND(SUM(ABS(consumption)), 3) as total_consumption,

       ROUND(AVG(peak_demand), 2) as avg_peak_demand,

       ROUND(AVG(efficiency_rating), 2) as avg_efficiency,

       COUNT(DISTINCT meter_id) as active_meters

FROM energy.analytics.energy_readings

GROUP BY location

ORDER BY total_consumption DESC

""")



location_analysis.show()


# Energy type efficiency analysis

print("\n=== Energy Type Efficiency Analysis ===")

energy_efficiency = spark.sql("""

SELECT energy_type, COUNT(*) as total_readings,

       ROUND(AVG(ABS(consumption)), 3) as avg_consumption,

       ROUND(AVG(efficiency_rating), 2) as avg_efficiency,

       ROUND(MAX(peak_demand), 2) as max_peak_demand,

       COUNT(DISTINCT meter_id) as unique_meters

FROM energy.analytics.energy_readings

GROUP BY energy_type

ORDER BY avg_consumption DESC

""")



energy_efficiency.show()


# Daily consumption patterns

print("\n=== Daily Consumption Patterns ===")

daily_patterns = spark.sql("""

SELECT DATE(reading_date) as date, HOUR(reading_date) as hour,

       ROUND(SUM(ABS(consumption)), 3) as total_consumption,

       ROUND(AVG(peak_demand), 2) as avg_peak_demand,

       COUNT(*) as reading_count

FROM energy.analytics.energy_readings

WHERE DATE(reading_date) = '2024-02-01'

GROUP BY DATE(reading_date), HOUR(reading_date)

ORDER BY hour

""")



daily_patterns.show()


# Monthly consumption trends

print("\n=== Monthly Consumption Trends ===")

monthly_trends = spark.sql("""

SELECT DATE_FORMAT(reading_date, 'yyyy-MM') as month,

       ROUND(SUM(ABS(consumption)), 3) as monthly_consumption,

       ROUND(AVG(peak_demand), 2) as avg_peak_demand,

       ROUND(AVG(efficiency_rating), 2) as avg_efficiency,

       COUNT(DISTINCT meter_id) as active_meters

FROM energy.analytics.energy_readings

GROUP BY DATE_FORMAT(reading_date, 'yyyy-MM')

ORDER BY month

""")



monthly_trends.show()

=== Meter Performance Analysis ===


+---------+--------------+---------------+---------------+--------------+--------------------------+
| meter_id|total_readings|avg_consumption|max_peak_demand|avg_efficiency|total_absolute_consumption|
+---------+--------------+---------------+---------------+--------------+--------------------------+
|MTR001031|          2160|         217.68|        3438.64|         84.45|                618706.118|
|MTR001601|          2160|        212.481|        3290.42|         84.41|                615332.819|
|MTR000731|          2160|        208.168|        3384.62|         84.58|                613149.576|
|MTR000498|          2160|        218.411|        3269.85|         84.62|                  610811.8|
|MTR001677|          2160|        214.368|        3111.93|         84.52|                610499.368|
|MTR000756|          2160|        207.871|        3170.51|         84.52|                 609804.32|
|MTR000738|          2160|        212.499|        3368.23|         84.66|                60

+---------------+--------------+-----------------+---------------+--------------+-------------+
|       location|total_readings|total_consumption|avg_peak_demand|avg_efficiency|active_meters|
+---------------+--------------+-----------------+---------------+--------------+-------------+
| Industrial_HOU|        862832|  5.83683373747E8|         879.43|         90.51|         2000|
| Commercial_SFO|        862761|  2.22486257475E8|         335.26|          81.5|         2000|
| Commercial_CHI|        864849|  2.14778356813E8|         322.87|          81.5|         2000|
|Residential_NYC|        865090|   5.5338107161E7|          83.17|         84.25|         2000|
|Residential_LAX|        864468|   5.3089571892E7|          79.83|          84.5|         2000|
+---------------+--------------+-----------------+---------------+--------------+-------------+


=== Energy Type Efficiency Analysis ===


+-----------+--------------+---------------+--------------+---------------+-------------+
|energy_type|total_readings|avg_consumption|avg_efficiency|max_peak_demand|unique_meters|
+-----------+--------------+---------------+--------------+---------------+-------------+
|      Water|       1080209|          506.2|          84.0|        3450.33|         2000|
|Electricity|       1080461|        274.209|         83.99|        2765.99|         2000|
|      Solar|       1079655|        141.955|          82.8|        1705.38|         2000|
|Natural Gas|       1079675|        123.221|          87.0|         955.52|         2000|
+-----------+--------------+---------------+--------------+---------------+-------------+


=== Daily Consumption Patterns ===


+----------+----+-----------------+---------------+-------------+
|      date|hour|total_consumption|avg_peak_demand|reading_count|
+----------+----+-----------------+---------------+-------------+
|2024-02-01|   0|       451485.469|         293.78|         2000|
|2024-02-01|   1|       456081.564|         296.89|         2000|
|2024-02-01|   2|       186286.651|         121.16|         2000|
|2024-02-01|   3|       184992.821|         120.54|         2000|
|2024-02-01|   4|        188674.42|         122.68|         2000|
|2024-02-01|   5|       186561.449|         122.52|         2000|
|2024-02-01|   6|        984331.46|         640.24|         2000|
|2024-02-01|   7|       981030.505|         639.02|         2000|
|2024-02-01|   8|       975953.168|         633.09|         2000|
|2024-02-01|   9|       466503.579|         303.06|         2000|
|2024-02-01|  10|       445455.596|         289.59|         2000|
|2024-02-01|  11|       467970.723|         305.52|         2000|
|2024-02-0

+-------+-------------------+---------------+--------------+-------------+
|  month|monthly_consumption|avg_peak_demand|avg_efficiency|active_meters|
+-------+-------------------+---------------+--------------+-------------+
|2024-01|    4.04155944396E8|          353.1|         84.45|         2000|
|2024-02|    3.78452396762E8|         353.45|         84.45|         2000|
|2024-03|     3.4676732593E8|         313.08|         84.45|         2000|
+-------+-------------------+---------------+--------------+-------------+



## Key Takeaways: Delta Liquid Clustering in AIDP

### What We Demonstrated

1. **Automatic Optimization**: Created a table with `CLUSTER BY (meter_id, reading_date)` and let Delta automatically optimize data layout

2. **Performance Benefits**: Queries on clustered columns (meter_id, reading_date) are significantly faster due to data locality

3. **Zero Maintenance**: No manual partitioning, bucketing, or Z-Ordering required - Delta handles it automatically

4. **Real-World Use Case**: Energy analytics where smart grid monitoring and consumption analysis are critical

### AIDP Advantages

- **Unified Analytics**: Seamlessly integrates with other AIDP services
- **Governance**: Catalog and schema isolation for energy data
- **Performance**: Optimized for both OLAP and OLTP workloads
- **Scalability**: Handles energy-scale data volumes effortlessly

### Best Practices for Liquid Clustering

1. **Choose clustering columns** based on your most common query patterns
2. **Start with 1-4 columns** - too many can reduce effectiveness
3. **Consider cardinality** - high-cardinality columns work best
4. **Monitor and adjust** as query patterns evolve

### Next Steps

- Explore other AIDP features like AI/ML integration
- Try liquid clustering with different column combinations
- Scale up to larger energy datasets
- Integrate with real smart meter and IoT sensor data

This notebook demonstrates how Oracle AI Data Platform makes advanced energy analytics accessible while maintaining enterprise-grade performance and governance.